# 第11回 その3: ニューラルネットワークを使った手書き数字認識
ここでは，もう少し具体的なデータとして，手書き数字の識別を行います。  


## ステップ1: 手書き数字データの準備
まずは必要ライブラリをインポートします。

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

機械学習のscikit-learnライブラリには，いくつかサンプルデータが用意されています。  
ここでは，手書き数字データを使います。

In [ ]:
# scikit-learn ライブラリの手書き数字データのインポート
from sklearn.datasets import load_digits

# 手書き数字データを読み込む
digit_data=load_digits()

print(np.shape(digit_data.images))
print(np.shape(digit_data.target))

データ数は1797です。  
`digit_data.images`には各データの画像情報が格納されています。  
画像サイズは$8 \times 8$で，各点には画素の輝度値が格納されています。  
`digit_data.target`にはクラス(どの数字を描いているのか)情報が格納されています。  
例としてに，データ番号0～9までの`images`と`target`を表示してみます。


In [ ]:
plt.figure(figsize=(7, 7))

# 0番目のデータそのものを表示
print('images[0]:')
print(digit_data.images[0])

# 0～9番目のデータを図として表示
print('images:')
for n in range(10):
  plt.subplot(4, 3, n+1)
  plt.imshow(digit_data.images[n], cmap='gray')
plt.show()

# target を表示
print('target:')
print(digit_data.target[:10])

今回，データは画像なので2次元行列です。  
一方ニューラルネットワークの入力はベクトルなので，そのままだとニューラルネットワークに入力ができません。  
そこで，画像の各行を横に連結して，$8 \times 8 = 64$次元のベクトルに変換します。  
これは numpy の reshape 関数を使って行います。  
また，データの標準化と one-hot ベクトルの作成も行います。

In [ ]:
# データのサンプル数と画像の高さ，幅を得る
num_samples, height, width = np.shape(digit_data.images)

# 画像の2次元行列をベクトル化した際の次元数を計算
num_dimensions = height * width

# クラス数は0～9の10クラス
num_classes = np.max(digit_data.target) + 1

print('Nunber of samples: ' + str(num_samples))
print('Number of dimensions: ' + str(num_dimensions))
print('Number of classes: ' + str(num_classes))

# digit_dataの形を[1797, 8, 8] から [1797, 64]へ変えることで，
# 各サンプルの8x8次元画像を64次元ベクトルに変形する。
X = np.reshape(digit_data.images, [num_samples, num_dimensions])

# 標準化も行っておく。
X = (X - np.mean(X, axis=0)) / (np.std(X, axis=0)+1E-7)

# クラス情報 Y
Y = digit_data.target

# one-hotベクトルの作成
Y_onehot = np.zeros([num_samples, num_classes])
for n in range(num_samples):
  # クラス番号(y[n])に対応するインデクスを1にする
  Y_onehot[n, Y[n]] = 1

今までは単一のデータセットに対して学習・評価を行っていましたが，一般的なクラス識別タスクでは，データセットを学習データと評価データに分けて行います。  
（一般には評価データを事前に知ることは不可能なので，評価データは学習には使わないということです。）  
ここでは，1797サンプルのうち，300サンプルを評価データに使用して，残りの1497サンプルを学習データに使用することにします。

In [ ]:
# 先頭の300サンプルを評価データに使用
X_test = X[:300]
Y_test = Y[:300]
Y_onehot_test = Y_onehot[:300]

# 残りのサンプルを学習データに使用
X_train = X[300:]
Y_train = Y[300:]
Y_onehot_train = Y_onehot[300:]

num_samples_train = np.size(Y_train)
num_samples_test = np.size(Y_test)

print('Nunber of training samples: ' + str(num_samples_train))
print('Nunber of test samples: ' + str(num_samples_test))

以上でデータの準備は完了です。

## ステップ2: ニューラルネットワークによる手書き数字認識  
用意したデータを使ってニューラルネットワークの学習と評価を行います。  

まずはニューラルネットワークの実装です。

In [ ]:
def sigmoid(z):
  '''
      シグモイド関数
  '''
  return (1+np.exp(-1*z))**(-1)


def softmax(z):
  '''
      ソフトマックス関数
      z: 要素数=クラス数のベクトル
      y_hat: クラス毎の確率。要素数=クラス数のベクトル
  '''
  z_norm = z - np.max(z) # np.exp(z)のオーバーフロー対策
  return (np.exp(z_norm) / np.sum(np.exp(z_norm)))


def cross_entropy(y_hat, y_onehot):
  '''
      クロスエントロピー損失を計算する。
      y_hat: マルチクラス回帰によって推定された y
      y_onehot: 正解のクラスの確率が1，それ以外の確率を0とするベクトル
                (one-hotベクトル)
  '''
  return -1.0 * np.sum(y_onehot*np.log(y_hat))

ニューラルネットワークの実装は `11_02_neural_network2.ipynb` のものと同じです。

In [ ]:
def initialize(num_inp, num_out, num_mid, seed=0):
  '''
      初期設定
      num_inp: 入力層のノード数(データの次元数)
      num_out: 出力層のノード数(=識別したいクラス数)
      num_mid: 各中間層のノード数。リスト型で定義
               中間層が2層で，それぞれノード数が5, 3とする場合
               num_mid = [5, 3]とする。
      seed: 乱数のシード。デフォルト値は0
  '''
  # 中間層の数は num_mid の要素数
  num_mid_layers = len(num_mid)
  # トータルの層数 = 中間層の数 + 1 (出力層の分)
  num_layers = num_mid_layers + 1

  np.random.seed(seed) # 乱数シードの固定

  # 中間層の重み行列 W の設定（各層の重み行列がリスト型で格納）
  W = []
  b = []
  for layer in range(num_layers):
    # まず，各層への入力ノード数と出力ノード数を確認
    if layer == 0:
      # 最初の中間層は入力データが入るため，入力ノード数=num_inp(データの次元数)
      n_in = num_inp
    else:
      # それ以外の場合，入力ノード数=一つ前の中間層のノード数
      n_in = num_mid[layer-1]
    if layer == num_layers - 1:
      # 出力層の場合，出力ノード数=num_out(クラス数)
      n_out = num_out
    else:
      # それ以外の場合，出力ノード数=中間層のノード数
      n_out = num_mid[layer]
      
    # W のサイズは[出力ノード数x入力ノード数]．乱数に基づいて初期値を作成
    W_tmp = np.random.normal(0, 1/np.sqrt(n_in), (n_out, n_in))
    # b のサイズは[出力ノード数]
    b_tmp = np.zeros(n_out)

    # 重み行列リストに追加
    W.append(W_tmp)
    b.append(b_tmp)

  return W, b


def forward(x, W, b):
  '''
      ニューラルネットワークにデータを通す(フォワード処理)
      x: 入力データ(サイズ=num_inpのベクトル)
      W: 各中間層および出力層の重み行列をリスト化したもの
      b: 各中間層および出力層のバイアスベクトルをリスト化したもの
  '''
  # 層の数 = 重み行列リストの要素数
  num_layers = len(W)

  # 層に通す。
  # このとき，各層における活性化関数に通す前と通した後の計算結果をそれぞれ
  # g, h としてリスト形式で記録しておく
  g = [] # 非線形関数(活性化関数)を通す前
  h = [] # 活性化関数を通した後
  for layer in range(num_layers):
    if layer == 0:
      # 最初の中間層では，入力層の値(=入力データ x)との内積を取る
      g_tmp = np.dot(W[layer], x) + b[layer]
    else:
      # 以降の層では，一つ前の層の値(h[-1])との内積を取る
      g_tmp = np.dot(W[layer], h[-1]) + b[layer]
      
    if layer == num_layers - 1:
      # 出力層の場合はソフトマックス関数に通す
        h_tmp = softmax(g_tmp)
    else:
      # それ以外の場合はシグモイド関数に通す
        h_tmp = sigmoid(g_tmp)

    # リストに追加
    g.append(g_tmp)
    h.append(h_tmp)

  # g と h を返す。最終的な出力はh[-1]
  return g, h


def backward(x, y, W, h):
  '''
      勾配を計算する(バックワード処理)
      x: 入力データ(サイズ=num_inpのベクトル)
      y: 正解の出力データ(サイズ=num_outのベクトル)
      W: 各中間層および出力層の重み行列をリスト化したもの
      h: 各中間層および出力層の計算結果をリスト化したもの
  '''
  # 層の数 = 重み行列リストの要素数
  num_layers = len(W)
  
  # 勾配を計算
  grad_W = []
  grad_b = []
  # forのrange()の後ろの[::-1]は逆順の意味。つまりnum_layers-1～0へカウントダウン。
  for layer in range(num_layers)[::-1]:
    # まず，prev_grad = ∂L/∂g を求める。prev_gradは逆伝播される。
    # prev_gradは出力層とそれ以外で計算が異なる。
    if layer == num_layers - 1:
      # 出力層の場合
      prev_grad = h[-1] - y
    else:
      # それ以外の層の場合
      prev_grad = np.dot(prev_grad, W[layer+1])*(1 - h[layer])*h[layer]

    # prev_gradを使って勾配を計算する。
    if layer == 0:
      # 入力層の場合
      grad_W_tmp = np.dot(np.array([prev_grad]).T, np.array([x]))
    else:
      # それ以外の場合
      grad_W_tmp = np.dot(np.array([prev_grad]).T, np.array([h[layer-1]]))
    grad_b_tmp = prev_grad

    # リストの先頭に追加
    grad_W.insert(0, grad_W_tmp)
    grad_b.insert(0, grad_b_tmp)
  
  # grad_Wとgrad_bを返す
  return grad_W, grad_b


def update(lr, W, b, grad_W, grad_b):
  '''
      勾配降下法によるパラメータの更新
      lr: 学習率
      W, b: 各中間層および出力層の重み行列とバイアスベクトルをリスト化したもの
      grad_W, grad_b，各層の勾配をリスト化したもの
  '''
  # 層の数 = 重み行列リストの要素数
  num_layers = len(W)

  for layer in range(num_layers):
    W[layer] -= lr * grad_W[layer]
    b[layer] -= lr * grad_b[layer]
  
  return W, b

さて，学習と評価を行います。  
それぞれの処理に対して，学習データと評価データに分けて行っている点に注意してください。

まずは中間層の数が1，ノード数が20の3層ニューラルネットワークでの評価です。

In [ ]:
# 中間層のノード数
num_middle_node = [20]

# 学習率
lr = 0.2
# エポック数
num_epochs = 100

# 重み行列の初期化を行う
W, b = initialize(num_dimensions, num_classes, num_middle_node)


# 損失関数の履歴
loss_history = np.array([])
# 正解率の履歴
acc_train_history = np.array([])
acc_test_history = np.array([])

for epoch in range(num_epochs):
  
  # 学習データを用いてニューラルネットワークのパラメータを更新

  # データをシャッフルしなおす。
  # (局所解に陥りにくくなるため)
  shuffle_index = np.random.permutation(np.arange(num_samples_train))
  X_tmp = X_train[(shuffle_index)]
  Y_onehot_tmp = Y_onehot_train[(shuffle_index)]
  
  loss = 0
  acc_train = 0
  acc_test = 0
  for n in range(num_samples_train):
    x = X_tmp[n]
    y_onehot = Y_onehot_tmp[n]

    # ニューラルネットワークに与える
    g, h = forward(x, W, b)
    # 最終出力はリストhの最後の要素である
    y_hat = h[-1]
  
    # 勾配の計算
    grad_W, grad_b = backward(x, y_onehot, W, h)

    # 更新
    W, b = update(lr, W, b, grad_W, grad_b)

    # 損失関数の蓄積
    loss += cross_entropy(y_hat, y_onehot)
  
  # エポック毎の識別正解率を測る。
  # ここでは学習データ，評価データ両方に対して正解率を求めている。
  # 学習データ
  for n in range(num_samples_train):
    x = X_train[n]
    y = Y_train[n]

    # ニューラルネットワークに与える
    g, h = forward(x, W, b)
    # 最終出力はリストhの最後の要素である
    y_hat = h[-1]
    # 正解率の蓄積
    if np.argmax(y_hat) == y:
      acc_train += 1
  
  # 評価データ
  for n in range(num_samples_test):
    x = X_test[n]
    y = Y_test[n]

    # ニューラルネットワークに与える
    g, h = forward(x, W, b)
    # 最終出力はリストhの最後の要素である
    y_hat = h[-1]
    # 正解率の蓄積
    if np.argmax(y_hat) == y:
      acc_test += 1
  
  loss /= num_samples_train
  acc_train /= num_samples_train
  acc_test /= num_samples_test
  loss_history = np.append(loss_history, loss)
  acc_train_history = np.append(acc_train_history, acc_train*100)
  acc_test_history = np.append(acc_test_history, acc_test*100)
  print('%d-th epoch: cross entropy = %.3f, train_accuracy = %.3f%%, test_accuracy = %.3f%%' % (epoch+1, loss, acc_train*100, acc_test*100))


# クロスエントロピーおよび正解率のプロット
plt.figure(figsize=(14,5))
plt.subplot(1,2,1)
plt.plot(loss_history)
plt.xlabel('epoch')
plt.ylabel('loss (cross entropy)')
plt.subplot(1,2,2)
plt.plot(acc_train_history, label='training data')
plt.plot(acc_test_history, label='test data')
plt.xlabel('epoch')
plt.ylabel('accuracy [%]')
plt.legend()
plt.show()

学習データに対しては100%，評価データに対しては92%の正解率でした。  
評価データは学習データに含まれないデータですので，一般的には学習データの正解率に比べて悪くなります。

次に，中間層の数が2，ノード数が20, 10の4層ニューラルネットワークの評価です。

In [ ]:
# 中間層のノード数
num_middle_node = [20, 10]

# 学習率
lr = 0.2
# エポック数
num_epochs = 100

# 重み行列の初期化を行う
W, b = initialize(num_dimensions, num_classes, num_middle_node)


# 損失関数の履歴
loss_history = np.array([])
# 正解率の履歴
acc_train_history = np.array([])
acc_test_history = np.array([])

for epoch in range(num_epochs):
  
  # 学習データを用いてニューラルネットワークのパラメータを更新

  # データをシャッフルしなおす。
  # (局所解に陥りにくくなるため)
  shuffle_index = np.random.permutation(np.arange(num_samples_train))
  X_tmp = X_train[(shuffle_index)]
  Y_onehot_tmp = Y_onehot_train[(shuffle_index)]
  
  loss = 0
  acc_train = 0
  acc_test = 0
  for n in range(num_samples_train):
    x = X_tmp[n]
    y_onehot = Y_onehot_tmp[n]

    # ニューラルネットワークに与える
    g, h = forward(x, W, b)
    # 最終出力はリストhの最後の要素である
    y_hat = h[-1]
  
    # 勾配の計算
    grad_W, grad_b = backward(x, y_onehot, W, h)

    # 更新
    W, b = update(lr, W, b, grad_W, grad_b)

    # 損失関数の蓄積
    loss += cross_entropy(y_hat, y_onehot)
  
  # エポック毎の識別正解率を測る。
  # ここでは学習データ，評価データ両方に対して正解率を求めている。
  # 学習データ
  for n in range(num_samples_train):
    x = X_train[n]
    y = Y_train[n]

    # ニューラルネットワークに与える
    g, h = forward(x, W, b)
    # 最終出力はリストhの最後の要素である
    y_hat = h[-1]
    # 正解率の蓄積
    if np.argmax(y_hat) == y:
      acc_train += 1
  
  # 評価データ
  for n in range(num_samples_test):
    x = X_test[n]
    y = Y_test[n]

    # ニューラルネットワークに与える
    g, h = forward(x, W, b)
    # 最終出力はリストhの最後の要素である
    y_hat = h[-1]
    # 正解率の蓄積
    if np.argmax(y_hat) == y:
      acc_test += 1
  
  loss /= num_samples_train
  acc_train /= num_samples_train
  acc_test /= num_samples_test
  loss_history = np.append(loss_history, loss)
  acc_train_history = np.append(acc_train_history, acc_train*100)
  acc_test_history = np.append(acc_test_history, acc_test*100)
  print('%d-th epoch: cross entropy = %.3f, train_accuracy = %.3f%%, test_accuracy = %.3f%%' % (epoch+1, loss, acc_train*100, acc_test*100))


# クロスエントロピーおよび正解率のプロット
plt.figure(figsize=(14,5))
plt.subplot(1,2,1)
plt.plot(loss_history)
plt.xlabel('epoch')
plt.ylabel('loss (cross entropy)')
plt.subplot(1,2,2)
plt.plot(acc_train_history, label='training data')
plt.plot(acc_test_history, label='test data')
plt.xlabel('epoch')
plt.ylabel('accuracy [%]')
plt.legend()
plt.show()

学習データの正解率が100%，評価データの正解率が93%と少し精度が上がりました。  
（たまたまの可能性が高いですが…。）

最後に，中間層のノード数が20, 10, 2となる5層ニューラルネットワークを学習させ，ノード数2の中間層の値をプロットしてみます。  

In [ ]:
# 中間層のノード数
num_middle_node = [20, 20, 2]

# 学習率
lr = 0.2
# エポック数
num_epochs = 100

# 重み行列の初期化を行う
W, b = initialize(num_dimensions, num_classes, num_middle_node)


# 損失関数の履歴
loss_history = np.array([])
# 正解率の履歴
acc_train_history = np.array([])
acc_test_history = np.array([])

for epoch in range(num_epochs):
  
  # 学習データを用いてニューラルネットワークのパラメータを更新

  # データをシャッフルしなおす。
  # (局所解に陥りにくくなるため)
  shuffle_index = np.random.permutation(np.arange(num_samples_train))
  X_tmp = X_train[(shuffle_index)]
  Y_onehot_tmp = Y_onehot_train[(shuffle_index)]
  
  loss = 0
  acc_train = 0
  acc_test = 0
  for n in range(num_samples_train):
    x = X_tmp[n]
    y_onehot = Y_onehot_tmp[n]

    # ニューラルネットワークに与える
    g, h = forward(x, W, b)
    # 最終出力はリストhの最後の要素である
    y_hat = h[-1]
  
    # 勾配の計算
    grad_W, grad_b = backward(x, y_onehot, W, h)

    # 更新
    W, b = update(lr, W, b, grad_W, grad_b)

    # 損失関数の蓄積
    loss += cross_entropy(y_hat, y_onehot)
  
  # エポック毎の識別正解率を測る。
  # ここでは学習データ，評価データ両方に対して正解率を求めている。
  # 学習データ
  for n in range(num_samples_train):
    x = X_train[n]
    y = Y_train[n]

    # ニューラルネットワークに与える
    g, h = forward(x, W, b)
    # 最終出力はリストhの最後の要素である
    y_hat = h[-1]
    # 正解率の蓄積
    if np.argmax(y_hat) == y:
      acc_train += 1
  
  # 評価データ
  for n in range(num_samples_test):
    x = X_test[n]
    y = Y_test[n]

    # ニューラルネットワークに与える
    g, h = forward(x, W, b)
    # 最終出力はリストhの最後の要素である
    y_hat = h[-1]
    # 正解率の蓄積
    if np.argmax(y_hat) == y:
      acc_test += 1
  
  loss /= num_samples_train
  acc_train /= num_samples_train
  acc_test /= num_samples_test
  loss_history = np.append(loss_history, loss)
  acc_train_history = np.append(acc_train_history, acc_train*100)
  acc_test_history = np.append(acc_test_history, acc_test*100)
  print('%d-th epoch: cross entropy = %.3f, train_accuracy = %.3f%%, test_accuracy = %.3f%%' % (epoch+1, loss, acc_train*100, acc_test*100))


# クロスエントロピーおよび正解率のプロット
plt.figure(figsize=(14,5))
plt.subplot(1,2,1)
plt.plot(loss_history)
plt.xlabel('epoch')
plt.ylabel('loss (cross entropy)')
plt.subplot(1,2,2)
plt.plot(acc_train_history, label='training data')
plt.plot(acc_test_history, label='test data')
plt.xlabel('epoch')
plt.ylabel('accuracy [%]')
plt.legend()
plt.show()

元々$8 \times 8$の10クラスのデータの情報を，中間層で2次元に圧縮しているため，どうしても正解率は下がってしまいますが，一応学習は行えましたので，2次元のデータを可視化してみます。

In [ ]:
# 出力層の直前の層の g の値
g_train = np.zeros([num_samples_train, 2])
g_test = np.zeros([num_samples_test, 2])

for n in range(num_samples_train):
  x = X_train[n]
  # ニューラルネットワークに与える
  g, h = forward(x, W, b)
  # 出力層の直前の層のインデクスは -2 
  g_train[n,:] = g[-2]

for n in range(num_samples_test):
  x = X_test[n]
  # ニューラルネットワークに与える
  g, h = forward(x, W, b)
  # 出力層の直前の層のインデクスは -2 
  g_test[n,:] = g[-2]

plt.figure(figsize=(7,7))
color_list = ['red', 'blue', 'green', 'yellow', 'cyan', 'magenta', 'black', 'brown', 'orange', 'lightgreen']
for k in range(num_classes):
  plt.scatter(g_train[Y_train==k,0], g_train[Y_train==k,1], color=color_list[k], label=str(k))
  plt.scatter(g_test[Y_test==k,0], g_test[Y_test==k,1], color=color_list[k], marker='x')
plt.legend()
plt.show()

●印が学習データ，x印がテストデータです。  
このようにすると，「2」と「3」の分布が近いことや，「8」と「9」の分布が近いこと，各評価データがどの数字に間違えやすいかといったことが分かります。